In [17]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

pd.set_option('display.max_columns', None)


DATA = '/Users/Marcy_Student/Desktop/marcy/DA2025_Lectures_Kevin/Mod6/Mod6_Project/Data/Motor_Vehicle_Collisions_-_Crashes_20251209.csv'

df = pd.read_csv(DATA)

# Drop bad coordinates
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['LONGITUDE'], df['LATITUDE']),
    crs='EPSG:4326'
)

# Load NYC borough shapes
boros = gpd.read_file('/Users/Marcy_Student/Desktop/marcy/DA2025_Lectures_Kevin/Mod6/Mod6_Project/nybb_25d/nybb.shp')
boros = boros.to_crs('EPSG:4326')

# Spatial join: assign borough to each crash point
df = gpd.sjoin(gdf, boros[['BoroName', 'geometry']], how='left', predicate='within')

df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'],format="%d/%m/%Y", errors='coerce')
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M').dt.hour

nan_counts_per_column = df.isnull().sum()

df.dtypes


CRASH DATE                       datetime64[ns]
CRASH TIME                                int32
BOROUGH                                  object
ZIP CODE                                float64
LATITUDE                                float64
LONGITUDE                               float64
LOCATION                                 object
ON STREET NAME                           object
CROSS STREET NAME                        object
OFF STREET NAME                          object
NUMBER OF PERSONS INJURED                 int64
NUMBER OF PERSONS KILLED                  int64
NUMBER OF PEDESTRIANS INJURED             int64
NUMBER OF PEDESTRIANS KILLED              int64
NUMBER OF CYCLIST INJURED                 int64
NUMBER OF CYCLIST KILLED                  int64
NUMBER OF MOTORIST INJURED                int64
NUMBER OF MOTORIST KILLED                 int64
CONTRIBUTING FACTOR VEHICLE 1            object
CONTRIBUTING FACTOR VEHICLE 2            object
CONTRIBUTING FACTOR VEHICLE 3           

In [27]:
print(df.groupby('BoroName')['NUMBER OF PERSONS INJURED'].sum().reset_index().sort_values(by='NUMBER OF PERSONS INJURED',ascending=False))

        BoroName  NUMBER OF PERSONS INJURED
1       Brooklyn                      64278
3         Queens                      54396
0          Bronx                      32250
2      Manhattan                      28583
4  Staten Island                       9312


In [ ]:
df['time_of_day'] = 'Night'  # default
# 1AM - 5AM
df.loc[df['CRASH TIME'] < 6, 'time_of_day'] = 'Midnight'
# 6AM - 11AM
df.loc[(df['CRASH TIME'] >= 6) & (df['CRASH TIME'] < 12), 'time_of_day'] = 'Morning'
# 12PM - 6pm
df.loc[(df['CRASH TIME'] >= 12) & (df['CRASH TIME'] < 18), 'time_of_day'] = 'Afternoon'

df['time_of_day']

print(df.groupby('time_of_day')['NUMBER OF PERSONS INJURED'].sum().reset_index().sort_values(by='NUMBER OF PERSONS INJURED',ascending=False))


  time_of_day  NUMBER OF PERSONS INJURED
0   Afternoon                      66125
3       Night                      56174
2     Morning                      42992
1    Midnight                      26211


In [30]:
print(df.groupby(['time_of_day', 'BoroName'])['NUMBER OF PERSONS INJURED'].sum().reset_index().sort_values(by='NUMBER OF PERSONS INJURED',ascending=False))

   time_of_day       BoroName  NUMBER OF PERSONS INJURED
1    Afternoon       Brooklyn                      22629
16       Night       Brooklyn                      19270
3    Afternoon         Queens                      18223
18       Night         Queens                      15532
11     Morning       Brooklyn                      13949
13     Morning         Queens                      13084
0    Afternoon          Bronx                      10785
15       Night          Bronx                       9822
2    Afternoon      Manhattan                       9787
6     Midnight       Brooklyn                       8430
17       Night      Manhattan                       8394
8     Midnight         Queens                       7557
10     Morning          Bronx                       6997
12     Morning      Manhattan                       6008
5     Midnight          Bronx                       4646
7     Midnight      Manhattan                       4394
4    Afternoon  Staten Island  

In [41]:
# Create a scatter plot
import plotly.express as px
fig = px.scatter(
    df,
    x='CRASH TIME',
    y='NUMBER OF PERSONS INJURED',
    title="Number of Persons Injured by Hour of Day",
)
fig.update_layout(xaxis_title="Hour of Day (0–23)")
fig.show()